Automatic downloading of multiple preset files and zips from different sources

In [40]:
import requests
from bs4 import BeautifulSoup as bs
import urllib.parse as ur
import os
import patoolib
import shutil
import rootutils
import http.cookiejar
import http.cookies
import cloudscraper
import sys

def extract_from_archive(archive_path, extract_to):
    # Extract all .fxp files from the archive (all subdirectories)
    patoolib.extract_archive(archive_path, outdir=extract_to)
    i = 0
    for root, dirs, files in os.walk(extract_to):
        for file in files:
            if file.endswith('.fxp'):
                i += 1
    return i

In [127]:
import requests
from bs4 import BeautifulSoup as bs
import urllib.parse as ur
import os
import patoolib
import http.cookiejar
import cloudscraper
import sys

cookies_file = 'presetshare_cookies.txt'

if not os.path.exists(cookies_file):
    print(f"Cookie file '{cookies_file}' not found. Please add your cookies first.")
    sys.exit(1)

# Setup session with cookies
session = cloudscraper.create_scraper()
session.cookies = http.cookiejar.MozillaCookieJar(cookies_file)

try:
    session.cookies.load(ignore_discard=True)
    print(f"Loaded cookies from {cookies_file}")
except Exception as e:
    print(f"Error loading cookies: {e}")
    sys.exit(1)

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
}

# Get download links
base_url = 'https://presetshare.com'
download_page = ur.urljoin(base_url, 'presets?query=&instrument=7&page=1')

print(f"Accessing {download_page}...")
response = session.get(download_page, headers=headers)

if response.status_code != 200:
    print(f"Failed to access page: {response.status_code}")
    sys.exit(1)

soup = bs(response.text, 'html.parser')
# save page as html for debugging
with open('preset.html', 'w', encoding='utf-8') as f:
    f.write(response.text)
download_links = [a['href'] for a in soup.find_all('a', class_='download-button')]
full_links = [ur.urljoin(base_url, link) for link in download_links]

print(f"Found {len(full_links)} download links.")

# Download all files
save_path = './test'
downloaded_files = []

for i, link in enumerate(full_links):
    filename = os.path.join(save_path, f'presetshare_{i}.fxp')
    print(f"Downloading {link}...")

    try:
        response = session.get(link, headers=headers)
        if response.status_code == 200:
            with open(filename, 'wb') as f:
                f.write(response.content)
            downloaded_files.append(filename)
            print(f"Saved to {filename}")
        else:
            print(f"Failed with status {response.status_code}")
    except Exception as e:
        print(f"Error downloading: {e}")

Loaded cookies from presetshare_cookies.txt
Accessing https://presetshare.com/presets?query=&instrument=7&page=1...
Found 24 download links.
Saved to ./test/presetshare_0.fxp
Saved to ./test/presetshare_1.fxp
Saved to ./test/presetshare_2.fxp
Saved to ./test/presetshare_3.fxp
Saved to ./test/presetshare_4.fxp
Saved to ./test/presetshare_5.fxp
Saved to ./test/presetshare_6.fxp
Saved to ./test/presetshare_7.fxp
Saved to ./test/presetshare_8.fxp
Saved to ./test/presetshare_9.fxp
Saved to ./test/presetshare_10.fxp
Failed with status 500
Saved to ./test/presetshare_12.fxp
Saved to ./test/presetshare_13.fxp
Failed with status 500
Failed with status 500
Saved to ./test/presetshare_16.fxp
Saved to ./test/presetshare_17.fxp
Saved to ./test/presetshare_18.fxp
Saved to ./test/presetshare_19.fxp
Saved to ./test/presetshare_20.fxp
Saved to ./test/presetshare_21.fxp
Saved to ./test/presetshare_22.fxp
Saved to ./test/presetshare_23.fxp
